In [2]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

## Part II: Estimation of Sensitivities in MC

In [257]:
@jit(nopython=False, parallel=True)
def European_Put_Delta(T, K, r, S, sigma, trials, e, seed_target=None):
    '''
    This function calculates the value of an European Put option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    Se = S + e
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    Se_adjust = Se * np.exp(r - (0.5 * sigma**2)*T)
    
    delta_array = np.zeros(trials)
    
    for i in range(trials):
        if seed_target != None:
            np.random.seed(seed=seed_target)
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        Se_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if K-S_cur > 0:
            payoff = (K-S_cur)*np.exp(-r*T)
        else:
            payoff = 0
            
        if K-Se_cur > 0:
            payoff_e = (K-Se_cur)*np.exp(-r*T)
        else:
            payoff_e = 0
            
        delta_array[i] = (payoff_e - payoff)/e

    return delta_array

In [304]:
T = 1
K = 99
r=0.06
S=100
sigma=0.2
trials=10**7
e=0.01
seed_target=None
np.mean(European_Put_Delta(T,K,r,S,sigma,trials,e,seed_target))

-0.2827054451840649